# Sudoku Solver explained

**squares** = A1

**unit** = collection of nine squares, being either one full row, column, or box

**peers (vecinos)** = squares that share a unit

## Reglas
- El tablero tiene 9x9 posiciones.
- Solo puedes usar valores del 1 al 9
- Cada subcuadrante de 3x3 puede contener solo una vez un valor del 1 al 9
- Cada columna puede contener solo una vez un valor del 1 al 9
- Cada fila puede contener solo una vez un valor del 1 al 9

In [592]:
import time

def cross(A, B):
    """ Producto cruz entre A y B. Para hacerlo, iterar en A, y dentro de cada iteración de A iterar 
    en B. En el segundo nivel de profundidad es donde se realiza la multiplicación """
    return [a+b for a in A for b in B]

digits   = '123456789'
rows     = 'ABCDEFGHI'
cols     = digits
print([c for c in cols])
# Hacemos el producto cruz entre filas y columnas para obtener los 81 valores dentro del sudoku
squares  = cross(rows, cols)
# Hacemos la lista de unidades formando 27 grupos. Esta nos ayudará a identificar las posibles permutaciones
unitlist = ([cross(rows, c) for c in cols] +
            [cross(r, cols) for r in rows] +
            [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')])
# Formamos un objeto que identifique como propiedad el square a identificar, y cada una de las listas 
# que contienen este valor.
units = dict((s, [u for u in unitlist if s in u]) for s in squares)
# Formamos una lista que indique todos los squares con los que un square puede estar dentro de la misma
# fila, columna, o subcuadrante.
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in squares)

['1', '2', '3', '4', '5', '6', '7', '8', '9']


In [593]:
# Pruebas para determinar que el sudoku tenga una configuración valida.
def test():
    """ Probar que hemos formado las correctas células que nos ayurdarán a resolver el sudoku """
    # Verificar que el sudoku thenga 81 cuadrados (grid 9x9).
    assert len(squares) == 81
    
    # Verificar que haya 27 unidades posibles (cada cuadrante (9) mas las filas (9) y las 
    # columnas (9) de cada uno).
    assert len(unitlist) == 27
    
    # Verificar que cada unidad tenga tres elementos con quien esté comparando (cuadrante, fila y 
    # columna).
    assert all(len(units[s]) == 3 for s in squares)
    
    # Verificar que haya 20 elementos que compartan espacio con un square. Es decir, los del
    # subcuadrante, y los de la fila y la columna de ese valor. 
    assert all(len(peers[s]) == 20 for s in squares)
    
    # Ejemplo manual que comprueba que dentro de las siguientes filas se encuentre el valor 'C2', y que
    # este a su vez sea el valor que se encuentre en la unidad C2
    assert units['C2'] == [['A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2'],
                           ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9'],
                           ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']]
    
    # Verificar que 'C2' comparta espacio con los siguientes valores. Numeramos el subcuadrante
    # y los valores restantes en la misma fila y columna de C2. A su vez, que este sea el valor 
    # contenido en el objeto peers que producimos para 'C2'
    assert peers['C2'] == set(['A2', 'B2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2',
                               'C1', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9',
                               'A1', 'A3', 'B1', 'B3'])
    # Si todas las pruebas pasan, imprimirlo.
    print('All tests pass.')
    
test()

All tests pass.


In [594]:
def parse_grid(grid):
    """ Leemos un grid a partir de una simple linea de texto. Asignamos a cada square un valor, usando
    un dict (JS object) para mapear a cada square con su valor, o con un conjunto de sus posibles
    valores """
    # Damos todos los posibles valores a cada square.
    values = dict((s, digits) for s in squares)

    # Después iteramos en el dict para asignar el valor dado en el input al square, mientras a su vez
    # comprobamos que un valor no tenga una incorrecta asignación con base en las reglas del sudoku (Propagación
    # de restricciones). Si es el caso, regresamos false a la solución.
    for s,d in grid_values(grid).items():
        if d in digits and not assign(values, s, d):
            return False #Falla si no podemos asignar un (d)ígito a un (s)quare
    return values

In [595]:
def grid_values(grid):
    """ Representar el sudoku en un diccionario en donde la key es el square y el valor es un 
    número o una unidad vacía (representado por '.' o '0'). """
    # chars = [c for c in grid if c in digits or c in '0.']
    chars = []
    for c in grid:
        if c in digits or c in '0.':
            chars.append(c)

    
    # Confirmamos que la nueva representación siga teniendo el mismo tamaño (grid 9x9).
    assert len(chars) == 81
    
    # Regresa los valores en un diccionario de tuplas.
    return dict(zip(squares, chars))

# Propagación de restricciones
Cuando encontramos una de los dos posibles restricciones que nos ayudarán a resolver el sudoku i.e.
1. Si un square tiene solo un posible valor, lo eliminamos de sus vecinos
2. Si una unidad tiene sólo un lugar posible para un valor, ponemos el valor

Esto se propaga para crear mas situaciones de (1) y (2), asignando valores con base en las restricciones que tiene cada lugar.

In [596]:
def assign(values, s, d):
    """ Eliminar todos los valores de cada square, excepto su dígito asignado """
    # Reemplazamos todos los dígitos con un espacio vacío 
    other_values = values[s].replace(d, '')
    # Si todos los squares fueron resueltos, regresamos el grid, de lo contrario False
    if all(eliminate(values, s, d2) for d2 in other_values):
        return values
    else:
        return False

In [597]:
def eliminate(values, s, d):
    """ Eliminar el (d)ígito del valor de otros (s)quares. Propagar cuando los valores o los lugares son menor o igual a 2. 
    Regresar falso si una contradicción es encontrada. Las contradicciones se definen adentro
    values = dict con todos los squares y sus valores
    s      = el cuadrante del que se quiere eliminar el valor.
    d      = el numero o valor que se quiere eliminar. """

    # Si el (d)ígito no esta en nuestro (s)quare, ya lo eliminamos previamente
    if d not in values[s]:
        return values 
    
    # Se reemplaza el valor d con un espacio vacío.
    values[s] = values[s].replace(d,'')
    
    # (1) REGLA: 
    # CONTRADICCIÓN No se puede eliminar un valor si la lista ya esta vacía
    if len(values[s]) == 0:
        return False
    # Si el (s)quare es reducido a un solo valor d2 (desde assign()), eliminamos ese valor de sus vecinos. 
    elif len(values[s]) == 1:
        # En este escenario d2 es el único valor restante. Por lo que hay que seguir la regla (1)
        d2 = values[s]
        # Y hacemos llamada recursiva para seguir buscando restricciones en los vecinos del (s2)quare con (d2)ígito en función
        if not all(eliminate(values, s2, d2) for s2 in peers[s]):
            return False
    
    # (2) REGLA: 
    # Si una unidad tiene sólo un lugar posible para un valor, iteramos hasta encontrar donde colocar este valor.
    for u in units[s]:
        # Guardar las posibles coordenadas válidas para d.
        dplaces = [s for s in u if d in values[s]]
        
        # CONTRADICCIÓN Aqui no hay posibles lugares validos para d.
        if len(dplaces) == 0:
            return False
        # Si encontramos que la relga (2) se cumple i.e. solo hay una coordenada válida, asignamos el valor
        elif len(dplaces) == 1:
            # Y hacemos llamada recursiva para seguir buscando restricciones
                if not assign(values, dplaces[0], d):
                    return False
                
    # Regresar la nueva configuracón de los cuadrantes
    return values

In [598]:
def display(values):
    "Mostramos los valores en una matriz 2D"
    # Obtenemos la longitud mas larga de todos los squares
    maxWidth = max(len(values[s]) for s in squares)
    # Hardcode la separación horizontal por formato de lista.
    if maxWidth >= 2:
        line = [('|' if c in '36' else '-'*maxWidth) for c in cols]
    else:
        line = ['-----------|','-----------|', '-----------']
        
    for r in rows:
        # Imprimimos el valor dentro de 'values'. Si es la tercer o sexta columna, imprimimos un separador '|'
        print([''.join(values[r+c]+('|' if c in '36' else ''))for c in cols])
        # Si es la tercer o sexta fila, imprimos una fila separadora
        if r in 'CF': print(line)
    print

# SEARCH AND SOLVE

In [599]:
# Resolver sudoku

# Si la propagación de contradicciones falla en encontrar la solución. C
# Si se encuntra un error entonces regresar y considerar otros valores validos (de d en s)

def solve(grid): return search(parse_grid(grid))

In [600]:
def search(values):
    "Usamos búsqueda en profundidad."
    # No encontramos solución y seguimos el depth-first search
    if values is False:
        return False
    # Si todos los (s)quares tienen un solo elemento, terminamos!
    if all(len(values[s]) == 1 for s in squares): 
        return values 
    # Buscamos el (s)quare sin resolver con menos posibilidades i.e. we choose '134' over '236789'
    n,s = min((len(values[s]), s) for s in squares if len(values[s]) > 1)
    # Buscamos la solución con una nueva copia de values para hacer varias búsquedas al mismo tiempo sin afectar
    # el grid actual. Si un (s)quare fue resuelto, lo regresamos y seguimos búsqueda.
    return some(search(assign(values.copy(), s, d)) for d in values[s])

In [601]:
def some(seq):
    """ Función OR. Determinar si una opción fue asignada correctamente """
    for e in seq:
        if e: return e
    return False

In [602]:
def time_solve(grid):
    """ Función para determinar el tiempo que tardó un resolverse un sudoku """
    # Inicializamos un contador de performance i.e. un contador con la más alta resolución disponible para medir 
    # una duración corta 
    start = time.perf_counter()
    values = solve(grid)
    # Marcamos cuánto se tardo restando el start del actual.
    t = time.perf_counter()-start

    print('#########          Grid a solucionar    ##########')
    display(grid_values(grid))
    if values: 
        print('#########          ✅Solucionado✅       ##########')
        display(values)
    print('Tiempo=(%.2f seconds)\n' % t)

In [603]:
grid2 = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
# print('######### Solo con propagación de restricciones #########')
# display(parse_grid(grid2))
print('#########           Con depth-first search      ##########')
time_solve(grid2)

#########           Con depth-first search      ##########
#########          Grid a solucionar    ##########
['4', '.', '.|', '.', '.', '.|', '8', '.', '5']
['.', '3', '.|', '.', '.', '.|', '.', '.', '.']
['.', '.', '.|', '7', '.', '.|', '.', '.', '.']
['-----------|', '-----------|', '-----------']
['.', '2', '.|', '.', '.', '.|', '.', '6', '.']
['.', '.', '.|', '.', '8', '.|', '4', '.', '.']
['.', '.', '.|', '.', '1', '.|', '.', '.', '.']
['-----------|', '-----------|', '-----------']
['.', '.', '.|', '6', '.', '3|', '.', '7', '.']
['5', '.', '.|', '2', '.', '.|', '.', '.', '.']
['1', '.', '4|', '.', '.', '.|', '.', '.', '.']
#########          ✅Solucionado✅       ##########
['4', '1', '7|', '3', '6', '9|', '8', '2', '5']
['6', '3', '2|', '1', '5', '8|', '9', '4', '7']
['9', '5', '8|', '7', '2', '4|', '3', '1', '6']
['-----------|', '-----------|', '-----------']
['8', '2', '5|', '4', '3', '7|', '1', '6', '9']
['7', '9', '1|', '5', '8', '6|', '4', '3', '2']
['3', '4', '6|', '9', '1